In [1]:
# load the modules and functions

import sys
sys.path.insert(0, '../functions/')

import xarray as xr
import numpy as np

from adjust_lon_xr_dataset import adjust_lon_xr_dataset
from storm_interstorm_periods import *

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# remove known warnings 

import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

#### Calculate storm and interstorm statistics:

- mean heat fluxes
- mean air-sea temperature gradient
- mean relative humidity
- mean wind speed
- storm and interstorm periods
- number of interstorm periods in a season that were consecutively longer than 4 days


In [3]:
# run the code that calculates the storm and interstorm days in the season

from glob import glob
from tqdm.notebook import tqdm 

data_directory_lhf = glob('/Volumes/LaCie/Work/data/era5/DJF_1981_2023_daily_means/latent_heat_flux*.nc')
data_directory_shf = glob('/Volumes/LaCie/Work/data/era5/DJF_1981_2023_daily_means/sensible_heat_flux*.nc')
data_directory_ssr = glob('/Volumes/LaCie/Work/data/era5/DJF_1981_2023_daily_means/net_solar_radiation*.nc')
data_directory_str = glob('/Volumes/LaCie/Work/data/era5/DJF_1981_2023_daily_means/net_thermal_radiation*.nc')
data_directory_ws  = glob('/Volumes/LaCie/Work/data/era5/DJF_1981_2023_daily_means/winds*.nc')
data_directory_sic = glob('/Volumes/LaCie/Work/data/era5/DJF_1981_2023_daily_means/sea_ice_cover*.nc')

data_directory = sorted(data_directory_lhf + data_directory_shf + data_directory_ssr + data_directory_str + data_directory_ws + data_directory_sic)

years = np.arange(1981, 2023, 1)

storm_period = np.ndarray([years.size, 201, 1440]) * np.NaN
interstorm_period = np.ndarray([years.size, 201, 1440]) * np.NaN

storm_ws = np.ndarray([years.size, 201, 1440]) * np.NaN   
interstorm_ws = np.ndarray([years.size, 201, 1440]) * np.NaN   

storm_shf = np.ndarray([years.size, 201, 1440]) * np.NaN   
interstorm_shf = np.ndarray([years.size, 201, 1440]) * np.NaN   

storm_lhf = np.ndarray([years.size, 201, 1440]) * np.NaN   
interstorm_lhf = np.ndarray([years.size, 201, 1440]) * np.NaN

storm_str = np.ndarray([years.size, 201, 1440]) * np.NaN   
interstorm_str = np.ndarray([years.size, 201, 1440]) * np.NaN

storm_ssr = np.ndarray([years.size, 201, 1440]) * np.NaN   
interstorm_ssr = np.ndarray([years.size, 201, 1440]) * np.NaN

interstorm_period_total_days = np.ndarray([years.size, 201, 1440]) * np.NaN
storm_period_total_days = np.ndarray([years.size, 201, 1440]) * np.NaN   

for y in tqdm(range(years.size)):

    # decide on the year to analyse
    year = years[y]

    # list the files for that given year
    flist = [file for file in data_directory if file.endswith(str(year) + '_DJF.nc')]
    
    # read the files in with xarray
    ds = xr.open_mfdataset(flist)

    # load them into memory
    ds = ds.load()

    # remove the data points where there is sea ice

    # Create a mask for sea ice concentration values less than 0.15
    date_mask = ds.siconc < 0.15

    # Create a replacement array with NaN values for the same shape as ds
    replacement_array = xr.full_like(ds.siconc, fill_value=np.nan, dtype=float)

    # Combine the original data and the replacement array
    ds = replacement_array.combine_first(ds.where(date_mask))

    # Calculate relative humidity (RH) using the formula and add it to the input dataset
    
    # Dp = ds.d2m.data - 273.15 # dew point temperature
    # T  = ds.t2m.data - 273.15 # temperature in kelvin 
    
    # ds['rh'] = (('time', 'latitude', 'longitude'), 100 * (np.exp((17.625 * Dp) / (243.04 + Dp)) / np.exp((17.625 * T) / (243.04 + T))))  
    # ds['sh'] = (('time', 'latitude', 'longitude'), calculate_specific_humidity(T, ds['rh'].data))

    # determine the storm and interstorm periods
    
    for i, ln in enumerate(ds.longitude.data):
    
        for ii, lt in enumerate(ds.latitude.data):

            if (np.isnan(ds.siconc.sel(longitude=ln, latitude=lt).data)==False).sum() == ds.siconc.sel(longitude=ln, latitude=lt).data.size: # if 80% of the data is ice free, we analyse the whole dataset
    
                ds_gridcell = ds.sel(longitude=ln, latitude=lt)
                
                storm_indices, interstorm_indices = storm_interstorm_id(ds_gridcell['ws'].data, threshold=10)
        
                storm_period[y, ii, i] = np.median([len(s) for s in storm_indices])
                interstorm_period[y, ii, i] = np.median([len(s) for s in interstorm_indices])

                list_interstorm = np.array([len(s) for s in interstorm_indices]) 
                interstorm_period_total_days[y, ii, i] = list_interstorm.sum()

                list_storm = np.array([len(s) for s in storm_indices]) 
                storm_period_total_days[y, ii, i] = list_storm.sum()

                # define the storm and interstorm datasets

                storm_list = [item for sublist in storm_indices for item in sublist]
                interstorm_list = [item for sublist in interstorm_indices for item in sublist]

                ds_storm = ds_gridcell.isel(time=storm_list)
                ds_interstorm = ds_gridcell.isel(time=interstorm_list)

                storm_ws[y, ii, i] = ds_storm.ws.mean(dim='time')
                interstorm_ws[y, ii, i] = ds_interstorm.ws.mean(dim='time')

                storm_shf[y, ii, i] = ds_storm.sshf.mean(dim='time')
                interstorm_shf[y, ii, i] = ds_interstorm.sshf.mean(dim='time')

                storm_lhf[y, ii, i] = ds_storm.slhf.mean(dim='time')
                interstorm_lhf[y, ii, i] = ds_interstorm.slhf.mean(dim='time')

                storm_str[y, ii, i] = ds_storm.str.mean(dim='time')
                interstorm_str[y, ii, i] = ds_interstorm.str.mean(dim='time')

                storm_ssr[y, ii, i] = ds_storm.ssr.mean(dim='time')
                interstorm_ssr[y, ii, i] = ds_interstorm.ssr.mean(dim='time')    

  0%|          | 0/42 [00:00<?, ?it/s]

In [4]:
# Create xarray dataset
ds_storms = xr.Dataset(
    {
        "storm_period": (["time", "latitude", "longitude"], storm_period),
        "interstorm_period": (["time", "latitude", "longitude"], interstorm_period),
        
        "interstorm_period_total_days": (["time", "latitude", "longitude"], interstorm_period_total_days),     
        "storm_period_total_days": (["time", "latitude", "longitude"], storm_period_total_days),   
        
        "storm_ws": (["time", "latitude", "longitude"], storm_ws),             
        "interstorm_ws": (["time", "latitude", "longitude"], interstorm_ws),
        
        "storm_shf": (["time", "latitude", "longitude"], storm_shf),             
        "interstorm_shf": (["time", "latitude", "longitude"], interstorm_shf),        

        "storm_lhf": (["time", "latitude", "longitude"], storm_lhf),             
        "interstorm_lhf": (["time", "latitude", "longitude"], interstorm_lhf),                

        "storm_ssr": (["time", "latitude", "longitude"], storm_ssr),             
        "interstorm_ssr": (["time", "latitude", "longitude"], interstorm_ssr),                

        "storm_str": (["time", "latitude", "longitude"], storm_str),             
        "interstorm_str": (["time", "latitude", "longitude"], interstorm_str),                
          
    },
    coords={
        "time": years,
        "latitude": ds['latitude'],
        "longitude": ds['longitude'],
    },
)

In [5]:
# find the difference between storms and interstorms period

var = ['shf', 'lhf', 'ssr', 'str']

for v in var:

    ds_storms['diff_' + v] = (('time', 'latitude', 'longitude'), ds_storms['storm_' + v].data - ds_storms['interstorm_' + v].data)

ds_storms['diff_qnet'] = (('time', 'latitude', 'longitude'), ds_storms['diff_shf'].data + ds_storms['diff_lhf'].data + ds_storms['diff_ssr'].data + ds_storms['diff_str'].data)
ds_storms['storm_qnet'] = (('time', 'latitude', 'longitude'), ds_storms['storm_shf'].data + ds_storms['storm_lhf'].data + ds_storms['storm_ssr'].data + ds_storms['storm_str'].data)
ds_storms['interstorm_qnet'] = (('time', 'latitude', 'longitude'), ds_storms['interstorm_shf'].data + ds_storms['interstorm_lhf'].data + ds_storms['interstorm_ssr'].data + ds_storms['interstorm_str'].data)

In [26]:
ds_storms.to_netcdf('../../data/era5/era5_storm_interstorm_periods_1981_2023_DJF.nc')